In [1]:
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from sklearn.datasets import make_moons
from sklearn.model_selection import train_test_split
import numpy as np
import matplotlib.pyplot as plt

def create_three_layer_ann(input_dim=10, hidden_units=8, output_dim=1):
    """Create a three-layer artificial neural network."""
    model = Sequential([
        Dense(units=hidden_units, 
              activation='relu', 
              input_shape=(input_dim,),
              kernel_initializer='he_normal', 
              name="hidden_layer_1"),
        Dense(units=hidden_units // 2, 
              activation='relu', 
              kernel_initializer='he_normal', 
              name="hidden_layer_2"),
        Dense(units=output_dim, 
              activation='sigmoid' if output_dim == 1 else 'softmax', 
              kernel_initializer='glorot_normal', 
              name="output_layer")
    ])
    
    model.compile(optimizer='adam',
                 loss='binary_crossentropy' if output_dim == 1 else 'categorical_crossentropy',
                 metrics=['accuracy'])
    
    return model

def get_activations(model, X_sample):
    """Fetch activations for all layers in the model."""
    # Create a list to store activation models
    activation_models = []
    
    # Create a model for each layer
    for layer in model.layers:
        activation_models.append(
            tf.keras.Model(
                inputs=model.input,
                outputs=layer.output
            )
        )
    
    # Get activations for each layer
    activations = []
    for activation_model in activation_models:
        layer_output = activation_model.predict(X_sample)
        # Normalize activations for visualization
        layer_output = (layer_output - layer_output.min()) / (layer_output.max() - layer_output.min() + 1e-10)
        activations.append(layer_output)
    
    return activations

def visualize_ann(nodes, activations, ax):
    """Visualize a simple ANN with real activations."""
    ax.clear()
    
    # Define layer positions
    layer_x = [i * 2 for i in range(len(nodes))]
    max_nodes = max(nodes)
    
    # Draw nodes
    for i, num_nodes in enumerate(nodes):
        y_positions = np.linspace(0, max_nodes, num_nodes)
        for j, y in enumerate(y_positions):
            activation = activations[i][0][j]  # Access the first sample's activation
            circle = plt.Circle((layer_x[i], y), 0.3, 
                              color=plt.cm.viridis(activation), 
                              ec="k")
            ax.add_artist(circle)
            ax.text(layer_x[i], y, f"{activation:.2f}", 
                   ha='center', va='center', fontsize=8)
    
    # Draw connections
    for i in range(len(nodes) - 1):
        y_positions_1 = np.linspace(0, max_nodes, nodes[i])
        y_positions_2 = np.linspace(0, max_nodes, nodes[i + 1])
        for j, y1 in enumerate(y_positions_1):
            for k, y2 in enumerate(y_positions_2):
                ax.plot([layer_x[i], layer_x[i + 1]], 
                       [y1, y2], 'k-', lw=0.5, alpha=0.2)
    
    ax.set_xlim(-1, max(layer_x) + 1)
    ax.set_ylim(-1, max_nodes + 1)
    ax.axis('off')

def main():
    # Generate dataset
    X, y = make_moons(n_samples=500, noise=0.2, random_state=42)
    X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2, random_state=42)

    # Create and build the model
    model = create_three_layer_ann(input_dim=2, hidden_units=8, output_dim=1)
    
    # Train the model
    history = model.fit(X_train, y_train, 
                       epochs=5, 
                       batch_size=32, 
                       validation_data=(X_val, y_val),
                       verbose=1)

    # Select one sample and get activations
    X_sample = X_train[:1]
    activations = get_activations(model, X_sample)

    # Get number of neurons per layer
    nodes = [2] + [layer.output_shape[1] for layer in model.layers]  # Include input layer

    # Create visualization
    fig, ax = plt.subplots(figsize=(10, 6))
    visualize_ann(nodes, [np.array([[0.5, 0.5]])] + activations, ax)  # Add dummy input layer activations
    plt.title('Neural Network Architecture with Activations')
    plt.show()

    # Plot training history
    plt.figure(figsize=(10, 4))
    plt.subplot(1, 2, 1)
    plt.plot(history.history['loss'], label='Training Loss')
    plt.plot(history.history['val_loss'], label='Validation Loss')
    plt.title('Model Loss')
    plt.xlabel('Epoch')
    plt.ylabel('Loss')
    plt.legend()

    plt.subplot(1, 2, 2)
    plt.plot(history.history['accuracy'], label='Training Accuracy')
    plt.plot(history.history['val_accuracy'], label='Validation Accuracy')
    plt.title('Model Accuracy')
    plt.xlabel('Epoch')
    plt.ylabel('Accuracy')
    plt.legend()
    plt.tight_layout()
    plt.show()

if __name__ == "__main__":
    main()

Epoch 1/5


/Users/rithwanali/Documents/Backpropagation/Backpropagation-Visualizer/venv/lib/python3.12/site-packages/keras/src/layers/core/dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


13/13 ━━━━━━━━━━━━━━━━━━━━ 1s 9ms/step - accuracy: 0.7845 - loss: 0.6147 - val_accuracy: 0.8300 - val_loss: 0.5804
Epoch 2/5
13/13 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.7484 - loss: 0.6072 - val_accuracy: 0.8100 - val_loss: 0.5688
Epoch 3/5
13/13 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.7238 - loss: 0.6099 - val_accuracy: 0.8000 - val_loss: 0.5590
Epoch 4/5
13/13 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.7562 - loss: 0.5970 - val_accuracy: 0.8000 - val_loss: 0.5488
Epoch 5/5
13/13 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.7559 - loss: 0.5822 - val_accuracy: 0.8000 - val_loss: 0.5390


AttributeError: The layer sequential has never been called and thus has no defined input.